# Lab 6

In [ ]:
-- Monady -> Wykonują funkcję na opakowanej wartości, 
-- która zwraca opakowaną wartość
-- (>>=) -> Bierze monadę (Opakowaną wartość), funkcję zwracającą monadę i 
-- zwraca monadę (bind)
class Monad m where
    (>>=) :: m a -> (a -> m b) -> m b
    
instance Monad Maybe where
    Nothing >>= func = Nothing
    Just val >>= func  = func val

-- Przykład
half x = if even x
           then Just (x `div` 2)
           else Nothing
Just 20 >>= half >>= half >>= half --Nothing

-- Monada I/O
getLine >>= readFile >>= putStrLn
foo = do
    filename <- getLine
    contents <- readFile filename
    putStrLn contents

In [ ]:
-- (<$<)
(<$<) :: (a -> b) -> a -> b 
(<$<) = ($) 
(\x -> 2 * x + 1) <$< 1 --3

-- (>$>) 
(>$>) :: a -> (a -> b) -> b 
x >$> f = f x 
infixl 0 >$> 
1 >$> (\x -> 2 * x + 1) --3

-- (<.<) 
(<.<) :: (b -> c) -> (a -> b) -> (a -> c) 
(<.<) = (.) 
(+2) <.< (*3) <$< 5 --17

-- (>.>)
(>.>) :: (a -> b) -> (b -> c) -> (a -> c) 
f >.> g = g . f 
infixl 9 >.> 
(+2) >.> (*3) <$< 5 --21



In [ ]:
-- Maybe
safeTail :: [a] -> Maybe [a] 
safeTail []     = Nothing 
safeTail (x:xs) = Just xs 
safeTail [1,2] --Just 2

extractMaybe :: Maybe a -> a 
extractMaybe Nothing  = error "Nothing inside!" 
extractMaybe (Just x) = x 
[1,2] >$> safeTail >.> extractMaybe >.> safeTail --Just []


insertMaybe :: a -> Maybe a 
insertMaybe = Just 

-- (>^$>) 
(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b 
Nothing  >^$> _ = Nothing 
(Just x) >^$> f = f x 
infixl 1 >^$> 
insertMaybe [1,2] >^$> safeTail 

-- (>.>>) -> Kleisli composition 
(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c) 
f >.>> g = \x -> g (extractMaybe (f x)) 


safeDiv :: Int -> Int -> Maybe Int 
safeDiv x y | y /= 0    = Just $ x `div` y 
            | otherwise = Nothing 
 
safeF5 :: Int -> Int -> Int -> Maybe Int 
safeF5 x y z = 
  case (safeDiv 1000 x) of 
    Nothing -> Nothing 
    Just (iOverX) -> 
      case (safeDiv 100 y) of 
        Nothing -> Nothing 
        Just (iOverY) -> 
          case (safeDiv 10 z) of
            Nothing -> Nothing 
            Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ 

In [ ]:
-- Monada []
return 3 >>= (\x -> [1..x]) --[1,2,3]
[1,2] >>= (\x -> [-x,x]) --[-1,1,-2,2]
[1,2] >>= \x -> [-x,x] >>= \y -> return (x,y) --[(1,-1),(1,1),(2,-2),(2,2)]


doXs1 :: [(Int,Int,Int)] 
doXs1 = do 
  let xs = [1,2] 
  x <- xs 
  y <- xs 
  z <- xs 
  return (x,y,z) 
doXs1 --[(1,1,1),(1,1,2),(1,2,1),(1,2,2),(2,1,1),(2,1,2),(2,2,1),(2,2,2)]


In [ ]:
import Control.Monad 
xs2 :: [(Int,Int,Int)] 
xs2 = [ (x,y,z) | let xs = [1..3], x <- xs, y <- xs, z <- xs, x > y && y > z ] 
 
doXs2 :: [(Int,Int,Int)] 
doXs2 = do 
  let xs = [1..3] 
  x <- xs 
  y <- xs 
  z <- xs 
  guard $ x > y && y > z 
  return (x,y,z) 
 
doXs2' :: [(Int,Int,Int)] 
doXs2' = do 
  let xs = [1..3] 
  x <- xs 
  y <- xs 
  z <- xs 
  if x > y && y > z 
    then return (x,y,z) 
    else [] 
    

:t guard 
[1..10] >>= \x -> guard (even x) >> return x 
[1..10] >>= \x -> guard (even x) >>= \y -> return (x,y) 
[1..10] >>= \x -> return x
[1..10] >>= return 

# Zadania Quizy Lab 6

In [ ]:
-- Zad 1.
-- Przepisz bez do notacji
safeDiv x y = if y/=0 then Just (x `div` y) else Nothing

safeF x y z = do
    iOverX <- safeDiv 1000 x
    iOverY <- safeDiv 100 y
    let refVal = 550
        sum = iOverX + iOverY + z
    return $ max refVal sum
safeF 1 2 3

safeF' x y z = 
    safeDiv 1000 x 
        >>= \iOverY -> safeDiv 100 y 
            >>= \iOverX -> 
                let {refVal = 550; sum = iOverX + iOverY + z} 
                    in return $ max refVal sum
        
safeF' 1 2 3
:t fmap
-- Zad 2.
mapM::Monad m => (a->m b ) -> [a] -> m [b]
mapM f [] = return []
mapM f (x:xs) = do
    y<-f x
    ys<- mapM f xs
    return (y:ys)

mapM (Just . length) ["1","22","333"]

Just 2 : [Just 2, Just 3]

-- Zad 3
nonEmpty :: String -> Either String String
nonEmpty "" = Left "Empty!"
nonEmpty s = Right s

nonNegative :: Int -> Either String Int
nonNegative n = if n>=0 then Right n else Left "n<0"

data D1 = MkD1 {i::Int,s::String} deriving Show

validate :: D1 -> Either String D1
validate d = if (even (i d)) then Right d else Left "i is odd"

createD1 :: String -> Int -> Either String D1
createD1 s1 i1 = nonEmpty s1 >>= 
    \s-> nonNegative i1 >>= 
        \i-> validate $ MkD1 i s
    
createD1 "abc" 2
createD1 "abc" 1

## Zadania Quizy Wiki

In [ ]:
-- Zad 1
return (Left 1) >>= \x -> [Just x, Nothing] >>= \y -> return $ Right y
-- [Right (Just (Left 1)),Right Nothing]

-- Zad 2 
-- (Przepisać bez do)
-- do
--     l<-getLine
--     putStr "Checking the length..."
--     if length l<=10 then putStrLn " OK"
--     else putStrLn " Too long!"

-- getLine >>= \l -> putStr "Checking the length...">>if length l <=10 then
--     putStrLn "OK" else putStrLn "Too long"

-- Zad 3
Right ((+) 1) >>= \f -> ($) Right $ f 2
-- Right 3

-- Zad 4
return (Just [1,2]) >>= \x -> [Just x, Nothing]

-- Zad 5
-- data Maybe a = Nothing | Just a deriving Show
-- instance Monad Maybe where
--     (Just x) >>= g = g x
--     Nothing >>= _ = Nothing
--     (>>) = (*>)

-- Zad 6 (Przepisać bez do)

-- parseInt :: String -> Maybe (Int,[Char])
-- parseSep :: String -> Maybe ([Char],[Char])

-- parse2Ints :: String -> Maybe (Int,Int,[Char])
-- parse2Ints s = do
--     (i1,s1) <- parseInt s
--     (_,s2) <-parseSep s1
--     (i2,s3) <-parseInt s2
--     let minVal = 10
--     return (max i1 minVal, max i2 minVal, s3)

-- parse2Ints' s = parseInt s >>= \(i1,s1) -> 
--     parseSep s1 >>= \(_,s2) -> parseInt s2 >>= \(i2,i3) 
--         -> let minVal = 10 
--             in return (max i1 minVal, max i2 minVal, s3)

-- Zad 7
(>>=) [1..3] (\x -> [2*x,x] >>= \y -> [2*y])
-- [4,2,8,4,12,6]
